In [1]:
# Import Libraries
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sort.sort import *
%matplotlib inline

In [2]:
import string
import easyocr

In [3]:
# Initialize the OCR reader
reader = easyocr.Reader(["en"], gpu=False)

# Mapping dictionaries for character conversion
dict_char_to_int = {"O": "0", "I": "1", "J": "3", "A": "4", "G": "6", "S": "5"}

dict_int_to_char = {"0": "O", "1": "I", "3": "J", "4": "A", "6": "G", "5": "S"}


def write_csv(results, output_path):
    """
    Write the results to a CSV file.

    Args:
        results (dict): Dictionary containing the results.
        output_path (str): Path to the output CSV file.
    """
    with open(output_path, "w") as f:
        f.write(
            "{},{},{},{},{},{},{}\n".format(
                "frame_nmr",
                "car_id",
                "car_bbox",
                "license_plate_bbox",
                "license_plate_bbox_score",
                "license_number",
                "license_number_score",
            )
        )

        for frame_nmr in results.keys():
            for car_id in results[frame_nmr].keys():
                print(results[frame_nmr][car_id])
                if (
                    "car" in results[frame_nmr][car_id].keys()
                    and "license_plate" in results[frame_nmr][car_id].keys()
                    and "text" in results[frame_nmr][car_id]["license_plate"].keys()
                ):
                    f.write(
                        "{},{},{},{},{},{},{}\n".format(
                            frame_nmr,
                            car_id,
                            "[{} {} {} {}]".format(
                                results[frame_nmr][car_id]["car"]["bbox"][0],
                                results[frame_nmr][car_id]["car"]["bbox"][1],
                                results[frame_nmr][car_id]["car"]["bbox"][2],
                                results[frame_nmr][car_id]["car"]["bbox"][3],
                            ),
                            "[{} {} {} {}]".format(
                                results[frame_nmr][car_id]["license_plate"]["bbox"][0],
                                results[frame_nmr][car_id]["license_plate"]["bbox"][1],
                                results[frame_nmr][car_id]["license_plate"]["bbox"][2],
                                results[frame_nmr][car_id]["license_plate"]["bbox"][3],
                            ),
                            results[frame_nmr][car_id]["license_plate"]["bbox_score"],
                            results[frame_nmr][car_id]["license_plate"]["text"],
                            results[frame_nmr][car_id]["license_plate"]["text_score"],
                        )
                    )
        f.close()

Using CPU. Note: This module is much faster with a GPU.


In [4]:
def license_complies_format(text):
    """
    Check if the license plate text complies with the required format.

    Args:
        text (str): License plate text.

    Returns:
        bool: True if the license plate complies with the format, False otherwise.
    """
    if len(text) != 7:
        return False

    if (
        (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys())
        and (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys())
        and (
            text[2] in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
            or text[2] in dict_char_to_int.keys()
        )
        and (
            text[3] in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
            or text[3] in dict_char_to_int.keys()
        )
        and (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys())
        and (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys())
        and (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys())
    ):
        return True
    else:
        return False


def format_license(text):
    """
    Format the license plate text by converting characters using the mapping dictionaries.

    Args:
        text (str): License plate text.

    Returns:
        str: Formatted license plate text.
    """
    license_plate_ = ""
    mapping = {
        0: dict_int_to_char,
        1: dict_int_to_char,
        4: dict_int_to_char,
        5: dict_int_to_char,
        6: dict_int_to_char,
        2: dict_char_to_int,
        3: dict_char_to_int,
    }
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_


def read_license_plate(license_plate_crop):
    """
    Read the license plate text from the given cropped image.

    Args:
        license_plate_crop (PIL.Image.Image): Cropped image containing the license plate.

    Returns:
        tuple: Tuple containing the formatted license plate text and its confidence score.
    """

    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper().replace(" ", "")

        if license_complies_format(text):
            return format_license(text), score

    return None, None


def get_car(license_plate, vehicle_track_ids):
    """
    Retrieve the vehicle coordinates and ID based on the license plate coordinates.

    Args:
        license_plate (tuple): Tuple containing the coordinates of the license plate (x1, y1, x2, y2, score, class_id).
        vehicle_track_ids (list): List of vehicle track IDs and their corresponding coordinates.

    Returns:
        tuple: Tuple containing the vehicle coordinates (x1, y1, x2, y2) and ID.
    """
    x1, y1, x2, y2, score, class_id = license_plate

    foundIt = False
    for j in range(len(vehicle_track_ids)):
        xcar1, ycar1, xcar2, ycar2, car_id = vehicle_track_ids[j]

        if x1 > xcar1 and y1 > ycar1 and x2 < xcar2 and y2 < ycar2:
            car_indx = j
            foundIt = True
            break

    if foundIt:
        return vehicle_track_ids[car_indx]

    return -1, -1, -1, -1, -1

In [5]:
texts=[]

In [6]:
# Load Models
coco_model=YOLO('yolov8n.pt')
license_plate_detector=YOLO('best.pt')

In [7]:
results = {}
frame_nmr=-1

In [8]:
mot_tracker=Sort()

In [9]:
# Load Video
cap = cv2.VideoCapture(
    "/Users/atharvgulati/Desktop/Vehicle License Plate Detection/2103099-uhd_3840_2160_30fps.mp4"
)

In [10]:
vehicles=[2,3,5,7]

In [ ]:

ret = True
while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1) : int(y2), int(x1) : int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(
                    license_plate_crop, cv2.COLOR_BGR2GRAY
                )
                _, license_plate_crop_thresh = cv2.threshold(
                    license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV
                )

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(
                    license_plate_crop_thresh
                )

                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {
                        "car": {"bbox": [xcar1, ycar1, xcar2, ycar2]},
                        "license_plate": {
                            "bbox": [x1, y1, x2, y2],
                            "text": license_plate_text,
                            "bbox_score": score,
                            "text_score": license_plate_text_score,
                        },
                    }

# write results
write_csv(results, "./detections.csv")


0: 384x640 21 cars, 1 bus, 2 trucks, 66.1ms
Speed: 3.9ms preprocess, 66.1ms inference, 6.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 42.9ms
Speed: 1.4ms preprocess, 42.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 1 bus, 2 trucks, 77.5ms
Speed: 1.7ms preprocess, 77.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 78.9ms
Speed: 5.3ms preprocess, 78.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 43.9ms
Speed: 2.1ms preprocess, 43.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 40.3ms
Speed: 1.9ms preprocess, 40.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 1 truck, 42.9ms
Speed: 1.7ms preprocess, 42.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 37.

In [12]:
results

{0: {},
 1: {3.0: {'car': {'bbox': [751.0364816893489,
     1373.8685331854963,
     1425.3685002827242,
     1991.4130350239611]},
   'license_plate': {'bbox': [982.7545166015625,
     1781.5762939453125,
     1206.0250244140625,
     1847.5653076171875],
    'text': 'NA13NRU',
    'bbox_score': 0.4296318292617798,
    'text_score': 0.23233657369780797}}},
 2: {},
 3: {5.0: {'car': {'bbox': [2197.028951614282,
     1176.321194341989,
     2788.8906143751447,
     1758.5312274600626]},
   'license_plate': {'bbox': [2433.03271484375,
     1582.6458740234375,
     2588.982421875,
     1637.9583740234375],
    'text': 'NT51VSU',
    'bbox_score': 0.5001336336135864,
    'text_score': 0.08987478410016168}}},
 4: {5.0: {'car': {'bbox': [2191.6478577873,
     1176.8234715770823,
     2791.093639229352,
     1764.0777600250435]},
   'license_plate': {'bbox': [2427.992431640625,
     1584.5362548828125,
     2590.716552734375,
     1644.0462646484375],
    'text': 'HW51VSU',
    'bbox_score': 

In [13]:
texts

[]